In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from typing import List, Tuple

In [11]:
text = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [38]:
EMBEDDING_DIM = 10
CONTEXT_SIZE = 2

In [28]:
def get_trigrams(text: List[str]) -> List[Tuple]:
    """Get trigrams of our text
    
    Args:
        text: strings split by whitespace
        
    Returns:
        trigrams: ([ word_i-2, word_i-1 ], target word)
    """
    trigrams = []
    for i in range(len(text) - 2):
        gram = ([text[i], text[i+1]], text[i+2])
        trigrams.append(gram)
    return trigrams

In [29]:
trigrams = get_trigrams(text)

In [30]:
trigrams[:3]

[(['When', 'forty'], 'winters'),
 (['forty', 'winters'], 'shall'),
 (['winters', 'shall'], 'besiege')]

Create vocabulary and do simple tokenization.

In [31]:
vocab = set(text)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [41]:
class NGram(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGram, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [53]:
losses = []
loss_function = nn.NLLLoss()
model = NGram(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        optimizer.zero_grad()
        log_probs = model(context_idxs)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Train loss: {total_loss:.2f}')
    losses.append(total_loss)

Train loss: 522.91
Train loss: 520.40
Train loss: 517.91
Train loss: 515.43
Train loss: 512.98
Train loss: 510.54
Train loss: 508.11
Train loss: 505.70
Train loss: 503.30
Train loss: 500.92
